<a href="https://colab.research.google.com/github/ProtossDragoon/CoMoLab/blob/master/CV/Segmentation/bisenetv2_tensorflow_and_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvidia-smi

Thu Dec  3 00:52:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/gdrive/"My Drive"/WhitePaper
!git clone https://github.com/ProtossDragoon/bisenetv2-tensorflow.git
%ls

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-8828bcec6f92>", line 1, in <module>
    get_ipython().magic('cd /content/gdrive/"My Drive"/WhitePaper')
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-91>", line 2, in cd
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling 

OSError: ignored

In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf
assert tf.__version__.startswith('1')
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
%cd /content/gdrive/"My Drive"/WhitePaper/bisenetv2-tensorflow
%ls -al

# 프로그램 실행 시 반드시 bisenetv2-tensorflow 위치로 접근할 것.
# --weights_path 은 정확한 이름이 아니라 ckpt 까지만 지정해 주면 됨. xxxx of xxxx 파일도 반드시 필요함.
# --src_image_path 을 원래 이미지 하나만 지정해야 동작했는데, 경로로 지정하면 거기에 있는 모든 이미지를 전부 segmentation 하도록 코드를 편집함.
# test_bisenetv2_cityscapes.py 만 수정한 상태임.
!python3 ./tools/cityscapes/test_bisenetv2_cityscapes.py --weights_path ./model/cityscapes/bisenetv2/cityscapes.ckpt --src_image_path ./data/test_image/cityscapes

/content/gdrive/My Drive/WhitePaper/bisenetv2-tensorflow
total 62
drwx------ 2 root root 4096 Dec  1 04:11 bisenet_model/
drwx------ 2 root root 4096 Dec  1 04:11 config/
-rw------- 1 root root   26 Dec  1 04:11 _config.yml
drwx------ 2 root root 4096 Dec  1 04:11 data/
drwx------ 2 root root 4096 Dec  1 14:30 data_output/
drwx------ 2 root root 4096 Dec  1 04:11 data_provider/
drwx------ 2 root root 4096 Dec  1 05:56 .git/
-rw------- 1 root root 1530 Dec  1 04:11 .gitignore
drwx------ 2 root root 4096 Dec  1 14:30 .ipynb_checkpoints/
-rw------- 1 root root 1072 Dec  1 04:11 LICENSE
drwx------ 2 root root 4096 Dec  1 04:11 local_utils/
drwx------ 2 root root 4096 Dec  1 04:13 model/
-rw------- 1 root root 9492 Dec  1 04:11 README.md
-rw------- 1 root root  127 Dec  1 04:11 requirements.txt
drwx------ 2 root root 4096 Dec  1 04:11 scripts/
drwx------ 2 root root 4096 Dec  1 04:11 tools/
drwx------ 2 root root 4096 Dec  1 04:11 trainner/
-------version-------
tensorflow version 1.15.2
--

# Optimization

## Convert

### keras h5 model to frozen .pb model

In [ ]:
le import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.framework import graph_io


def keras_to_frozen_pb(model_in_path, 
                       model_out_path,
                       tensor_out_name=None,
                       tensorboard_dir=None):
    """
    Converter that transforms keras model to frozen pb model
    
    Args:
        model_in_path (str): Input model path (.h5) 
        model_out_path (str): Output model path (dir)
        tensor_out_name (str, optional): Specified name of output tensor. 
                                         If None, it will get default tensor name from keras model.
                                         Defaults to None.
        tensorboard_dir (str, optional): Output tensorboard dir path for inspecting output model graph.
                                         If None, it doesn't generate. 
                                         Defaults to None.
    """

    graph = tf.Graph()
    with graph.as_default():
        sess = tf.Session()
        K.set_session(sess)
        K.set_learning_phase(0)

        # load the model to graph and sess
        model = tf.keras.models.load_model(model_in_path)

        # get the tensor_out_name 
        if tensor_out_name is None:
            print('tensor_out_name was not selected manually. it is going to selected automatically.')
            if len(model.outputs) > 1:
                raise NameError("the model has multiple output tensor. Need to specify output tensor name.")
            else:
                print('model outptus : {}'.format(model.outputs))
                print('selected model outputs : {}'.format(model.outputs[0].name.split(':')[0]))
                tensor_out_name = model.outputs[0].name.split(":")[0]

        # freeze the graph
        # Turn all the variables into inline constants inside the graph and save it.

        # select one !
        # graph.as_graph_def() : method : Returns a serialized GraphDef representation of this graph.
        # sess.graph_def : attribute : A serializable version of the underlying TensorFlow graph.
        graphdef = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, [tensor_out_name])
        graphdef = tf.graph_util.remove_training_nodes(graphdef)

        #params : 
        # graph_or_graph_def
        # logdir : directory where to write the graph
        # name : filename for the graph
        graph_io.write_graph(graphdef, './', model_out_path, as_test = False)

    if not tensorboard_dir is None:
        tf.summary.FileWriter(logdir=tensorboard_dir, graph_def=graphdef)
    
    return tensor_out_name



input_keras_model = "modelpath"
output_pb_model = "outputpath"

model_out_name = keras_to_frozen_pb(input_keras_model, output_pb_model)
print("output node name : {}".format(model_out_name))

### frozen pb to tensorRT

In [ ]:
import tensorflow as tf
import uff
import tensorrt as trt

def frozen_pb_to_plan(model_path,
                      output_path,
                      tensor_in_name,
                      tensor_out_name,
                      input_size,
                      data_type=trt.float32,
                      max_batch_size=1,
                      max_workspace=1<<30,
                      tensorboard_dir=None):


    # -----
    # GFile : File I/O wrappers.
      # read more : https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile
    # read modelfile as binary format
    graph_def = tf.GraphDef()
    with tf.io.gfile.GFile(model_path, "rb") as f:
        graph_def.ParseFromString(f.read())
    
    # -----
    # uff : a set of utilites to convert trained models
    # Converts a TensorFlow frozen graph to a UFF model.
    # The uff package contains a set of utilites to convert trained models from various frameworks to a common format.
    # from_tensorflow_frozen_model returns serialized UFF MetaGraph (str)
      # read more : https://docs.nvidia.com/deeplearning/tensorrt/api/python_api/uff/uff.html
    uff_model = uff.from_tensorflow_frozen_model(model_path, [tensor_out_name])

    # create uff parser
    parser = trt.UffParser()
    parser.register_input(tensor_in_name, input_size)
    parser.register_output(tensor_out_name)

    # -----
    # create trt logger and builder
    trt_logger = tft.Logger(trt.Logger.INFO)
    trt_builder = trt.Builder(trt_logger)

    trt_builder.max_batch_size = max_batch_size
    trt_builder.max_workspace_size = max_batch_size
    trt_builder.fp16_mode = (data_type == trt.float16)

    # parse the uff model to trt builder
    network = tft_builder.create_network()
    parser.parse_buffer(uff_model, network)
    
    # build optimized inference engine
    engine = tft_builder.build_cuda_engine(network)

    # save inference engine
    with open(output_path, "wb") as f:
        f.write(engine.serialize())


model_path = 'pb model path'
output_path = 'output tensorrt plan path'
tensor_in_name = 'input node name'
tensor_out_name = 'output node name'
input_size = [width, height, channel] # order – Input order on which the framework input was originally.

frozen_pb_to_plan(model_path,
                  output_path,
                  tensor_in_name,
                  tensor_out_name,
                  input_size,
                  data_type=trt.float32,
                  max_batch_size=1,
                  max_workspace=1<<30,
                  tensorboard_dir=None)

SyntaxError: ignored